# Recommendations on GCP with TensorFlow and WALS
***
This lab is adapted from the original [solution](https://github.com/GoogleCloudPlatform/tensorflow-recommendation-wals) created by [lukmanr](https://github.com/GoogleCloudPlatform/tensorflow-recommendation-wals/commits?author=lukmanr) 

This project deploys a solution for a recommendation service on GCP, using the WALS algorithm in TensorFlow. Components include:

- Recommendation model code, and scripts to train and tune the model on ML Engine
- A REST endpoint using Google Cloud Endpoints for serving recommendations
- An Airflow server managed by Cloud Composer for running scheduled model training


## Confirm Prerequisites

### 1. Cloud Composer Instance
- Create a Cloud Composer [instance](https://console.cloud.google.com/composer/environments/create?project=)<br><br>

### 2. Ensure Cloud IAM permissions set for GAE
- In [IAM](https://console.cloud.google.com/iam-admin/iam?project=), change permissions for "Compute Engine default service account" from Editor to Owner. This is required so you can create and deploy App Engine versions from within Cloud Datalab<br><br>


### 3. Create Google App Engine instance
Uncomment and run only the below if you still need to create an App Engine instance:

In [ ]:
# run app engine creation commands
# %bash
# gcloud app create --region=[REGION] # see: https://cloud.google.com/compute/docs/regions-zones/
# gcloud app update --no-split-health-checks

### 4. Install required packages
Installing conda and requirements takes 3-5 minutes

In [35]:
%bash
cd endtoend
conda create -y -n recserve
source activate recserve
conda install -y -n recserve --file conda.txt

airflow       data		  README.md		   wals_ml_engine
app	      endtoend (2).ipynb  requirements.txt
conda.txt     LICENSE		  scripts
CONTRIBUTING  notebooks		  Untitled Notebook.ipynb


# Upload sample data to BigQuery 
This tutorial comes with a sample Google Analytics data set, containing page tracking events from the Austrian news site Kurier.at. The schema file '''ga_sessions_sample_schema.json''' is located in the folder data in the tutorial code, and the data file '''ga_sessions_sample.json.gz''' is located in a public Cloud Storage bucket associated with this tutorial. To upload this data set to BigQuery:

### 1. Make a GCS bucket with the name recserve_[YOUR-PROJECT-ID]:

In [ ]:
%bash 
export BUCKET=gs://recserve_$(gcloud config get-value project 2> /dev/null) # or specify your existing bucket
gsutil mb ${BUCKET}

# Copy the data file ga_sessions_sample.json.gz to the bucket:
gsutil cp gs://solutions-public-assets/recommendation-tensorflow/data/ga_sessions_sample.json.gz ${BUCKET}/data/ga_sessions_sample.json.gz

### 2. Create empty BigQuery dataset and load sample JSON data
Note: Ingesting the 400K rows of sample data usually takes 5-7 minutes

In [22]:
%bash
export PROJECT=$(gcloud config get-value project 2> /dev/null)
export BUCKET=gs://recserve_$(gcloud config get-value project 2> /dev/null)

bq --project_id=${PROJECT} mk GA360_test

bq load --source_format=NEWLINE_DELIMITED_JSON \
 GA360_test.ga_sessions_sample \
 ${BUCKET}/data/ga_sessions_sample.json.gz \
 data/ga_sessions_sample_schema.json


BigQuery error in mk operation: Dataset
'qwiklabs-gcp-77c0e3b62eaf4101:GA360_test' already exists.



Waiting on bqjob_r1485065c4c14fee3_00000166bf5d6382_1 ... (223s) Current status: DONE   

***
# Install WALS model training package and model data

### 1. Create a distributable package. Copy the package up to the code folder in the bucket you created previously.

In [23]:
%bash
export BUCKET=gs://recserve_$(gcloud config get-value project 2> /dev/null)
pushd wals_ml_engine
python setup.py sdist
gsutil cp dist/wals_ml_engine-0.1.tar.gz ${BUCKET}/code/

~/datalab/training-data-analyst/courses/machine_learning/deepdive/10_recommend/endtoend/wals_ml_engine ~/datalab/training-data-analyst/courses/machine_learning/deepdive/10_recommend/endtoend
running sdist
running egg_info
writing requirements to wals_ml_engine.egg-info/requires.txt
writing wals_ml_engine.egg-info/PKG-INFO
writing top-level names to wals_ml_engine.egg-info/top_level.txt
writing dependency_links to wals_ml_engine.egg-info/dependency_links.txt
reading manifest file 'wals_ml_engine.egg-info/SOURCES.txt'
writing manifest file 'wals_ml_engine.egg-info/SOURCES.txt'
running check
creating wals_ml_engine-0.1
creating wals_ml_engine-0.1/trainer
creating wals_ml_engine-0.1/wals_ml_engine.egg-info
copying files to wals_ml_engine-0.1...
copying README.md -> wals_ml_engine-0.1
copying setup.py -> wals_ml_engine-0.1
copying trainer/__init__.py -> wals_ml_engine-0.1/trainer
copying trainer/model.py -> wals_ml_engine-0.1/trainer
copying trainer/task.py -> wals_ml_engine-0.1/trainer
cop



Copying file://dist/wals_ml_engine-0.1.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  8.3 KiB/  8.3 KiB]                                                
Operation completed over 1 objects/8.3 KiB.                                      


### 2. Run the WALS model on the sample data set:

In [24]:
%bash
cd wals_ml_engine
pip install sh
./mltrain.sh local ../data/recommendation_events.csv --data-type web_views --use-optimized

Mon Oct 29 10:31:34 UTC 2018
Mon Oct 29 10:32:41 UTC 2018


/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Train Start: 2018-10-29 10:32:01
trainer/wals.py:94: RuntimeWarning: divide by zero encountered in divide
  frac = np.array(1.0/(data > 0.0).sum(axis))
2018-10-29 10:32:04.892281: W tensorflow/core/framework/allocator.cc:101] Allocation of 277358400 exceeds 10% of system memory.
2018-10-29 10:32:06.871211: W tensorflow/core/framework/allocator.cc:101] Allocation of 277358400 exceeds 10% of system memory.
2018-10-29 10:32:08.672524: W tensorflow/core/framework/allocator.cc:101] Allocation of 277358400 exceeds 10% of system memory.
2018-10-29 10:32:10.429864: W tensorflow/core/framework/allocator.cc:101] Allocation of 277358400 exceeds 10% of system memory.
2018

This will take a couple minutes, and create a job directory under wals_ml_engine/jobs like "wals_ml_local_20180102_012345/model", containing the model files saved as numpy arrays.

### 3. Copy the model files from this directory to the model folder in the project bucket:

In [25]:
%bash
cd wals_ml_engine
export BUCKET=gs://recserve_$(gcloud config get-value project 2> /dev/null)
export JOB_MODEL=$(find jobs -name "model" | tail -1)
gsutil cp ${JOB_MODEL}/* ${BUCKET}/model/

Copying file://jobs/wals_ml_local_20181029_101756/model/col.npy [Content-Type=application/octet-stream]...
Copying file://jobs/wals_ml_local_20181029_101756/model/item.npy [Content-Type=application/octet-stream]...
Copying file://jobs/wals_ml_local_20181029_101756/model/row.npy [Content-Type=application/octet-stream]...
Copying file://jobs/wals_ml_local_20181029_101756/model/user.npy [Content-Type=application/octet-stream]...
\ [4 files][ 10.5 MiB/ 10.5 MiB]                                                
Operation completed over 4 objects/10.5 MiB.                                     


### 4. Copy the sample data file up to the project bucket:

In [26]:
%bash
export BUCKET=gs://recserve_$(gcloud config get-value project 2> /dev/null)
gsutil cp data/recommendation_events.csv ${BUCKET}/data/
popd

Copying file://data/recommendation_events.csv [Content-Type=text/csv]...
-
Operation completed over 1 objects/10.0 MiB.                                     
bash: line 3: popd: directory stack empty


# Install the recserve endpoint

### 1. Create the App Engine app in your project:

In [28]:
# You should have already created an app engine account in Cloud Shell
# if not, run the below:
# %bash
# gcloud app create --region=us-east1
# gcloud app update --no-split-health-checks

### 2. Prepare the deploy template for the Cloud Endpoint API:

In [36]:
%bash
cd scripts
./prepare_deploy_api.sh                         # Prepare config file for the API.

Preparing config for deploying service in ../app/openapi.yaml...
To deploy:  gcloud endpoints services deploy /tmp/tmp.ovdaYKlHLs.yaml


This will output somthing like:

...
To deploy:  gcloud endpoints services deploy /var/folders/1m/r3slmhp92074pzdhhfjvnw0m00dhhl/T/tmp.n6QVl5hO.yaml

### 3. Run the endpoints deploy command output above:
Be sure to replace the below [FILE_NAME] with the results from above before running.

In [30]:
%bash
gcloud endpoints services deploy [REPLACE_WITH_TEMP_FILE_NAME.yaml]

Could not find conda environment: recserve
You can list all discoverable environments with `conda info --envs`.

ERROR: (gcloud.endpoints.services.deploy) Could not open service config file [[REPLACE_WITH_TEMP_FILE_NAME.yaml]]: Unable to read file [[REPLACE_WITH_TEMP_FILE_NAME.yaml]]: [Errno 2] No such file or directory: '[REPLACE_WITH_TEMP_FILE_NAME.yaml]'


### 4. Prepare the deploy template for the App Engine App:

In [38]:
%bash
cd scripts

# view available scripts
ls

# run prepare 
./prepare_deploy_app.sh

delete_project.sh
generate_traffic.sh
prepare_deploy_api.sh
prepare_deploy_app.sh
query_api_auth.sh
query_api.sh
util.sh
gcloud app create --region=us-east1
To deploy:  gcloud -q app deploy ../app/app_template.yaml_deploy.yaml


You are creating an app for project [qwiklabs-gcp-77c0e3b62eaf4101].
cannot be changed. More information about regions is at
<https://cloud.google.com/appengine/docs/locations>.

ERROR: (gcloud.app.create) The project [qwiklabs-gcp-77c0e3b62eaf4101] already contains an App Engine application. You can deploy your application using `gcloud app deploy`.


You can ignore the script output "ERROR: (gcloud.app.create) The project [...] already contains an App Engine application. You can deploy your application using gcloud app deploy." This is expected.

The script will output something like:

 ...
   To deploy:  gcloud -q app deploy app/app_template.yaml_deploy.yaml

### 5. Run the command above:

In [39]:
%bash
gcloud -q app deploy app/app_template.yaml_deploy.yaml

Services to deploy:

descriptor:      [/content/datalab/training-data-analyst/courses/machine_learning/deepdive/10_recommend/endtoend/app/app_template.yaml_deploy.yaml]
source:          [/content/datalab/training-data-analyst/courses/machine_learning/deepdive/10_recommend/endtoend/app]
target project:  [qwiklabs-gcp-77c0e3b62eaf4101]
target service:  [default]
target version:  [20181029t110558]
target url:      [https://qwiklabs-gcp-77c0e3b62eaf4101.appspot.com]


Beginning deployment of service [default]...
Building and pushing image for service [default]
Started cloud build [1fb20f84-22dd-47a0-bdd9-312f5f92266c].
To see logs in the Cloud Console: https://console.cloud.google.com/gcr/builds/1fb20f84-22dd-47a0-bdd9-312f5f92266c?project=941630459158
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "1fb20f84-22dd-47a0-bdd9-312f5f92266c"

FETCHSOURCE
Fetching storage object: gs://staging.qwiklabs-gcp-77c0e3b62eaf4101.appspot.com/us.gcr.io/qwi

This will take several minutes.

# Query the API
Lastly, you are able to test the recommendation model API by submitting a query request. Note the example userId passed and numRecs desired as the URL parameters for the model input.

In [40]:
%bash
cd scripts
./query_api.sh          # Query the API.
./generate_traffic.sh   # Send traffic to the API.

curl "https://qwiklabs-gcp-77c0e3b62eaf4101.appspot.com/recommendation?userId=5448543647176335931&numRecs=5"
{"articles":["299824032","299865757","299959410","299935287","299933565"]}



  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    75  100    75    0     0    270      0 --:--:-- --:--:-- --:--:--   270
